# Проект по SQL

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Цель** - проанализировать базу данных для составления ценностного предложения нового продукта. Для этого необходимо решить несколько нижепредставленных задач.

**Задачи:**
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Подключение к базе данных

In [1]:
import pandas as pd
from sqlalchemy import create_engine 

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db']) 

In [3]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Исследование таблиц

Выведем первые 5 строк каждой таблицы из нашей базы данных.

**Books**

In [4]:
query = '''
SELECT *
FROM books
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Authors**

In [5]:
query = '''
SELECT *
FROM authors
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Ratings**

In [6]:
query = '''
SELECT *
FROM ratings
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Reviews**

In [7]:
query = '''
SELECT *
FROM reviews
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Publishers**

In [8]:
query = '''
SELECT *
FROM publishers
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Задание №1

**Посчитайте, сколько книг вышло после 1 января 2000 года**

In [9]:
query = '''
SELECT COUNT(*)
FROM books
WHERE publication_date > '2000-01-01'
'''

In [10]:
pd.io.sql.read_sql(query, con = engine) 

,count
0,819


**Вывод:** 819 книг вышло после 1-го января.

## Задание №2

**Для каждой книги посчитайте количество обзоров и среднюю оценку.**

In [11]:
query = '''
SELECT *
FROM
    (SELECT *
     FROM
        (SELECT bs.book_id, bs.title, ROUND(AVG(rg.rating) OVER (PARTITION BY bs.book_id), 2) AS avg_rating, COUNT(DISTINCT(rv.review_id))  AS count_review
        FROM books as bs
        LEFT OUTER JOIN reviews AS rv ON bs.book_id = rv.book_id
        LEFT OUTER JOIN ratings AS rg ON bs.book_id = rg.book_id
        GROUP BY bs.book_id, rg.rating) as bh 
        ) as y
GROUP BY y.book_id, y.title, y.avg_rating, y.count_review

'''

In [12]:
pd.io.sql.read_sql(query, con = engine) 

,book_id,title,avg_rating,count_review
0,849,The Rescue,4.00,3
1,197,Dragonsinger (Harper Hall #2),3.00,2
2,595,Starship Troopers,4.50,3
3,165,Creepshow,4.50,1
4,882,The Terror,4.50,2
...,...,...,...,...
995,496,Out of the Silent Planet (The Space Trilogy #1),3.50,3
996,939,Tropic of Cancer,2.50,2
997,88,Assassin's Quest (Farseer Trilogy #3),3.67,3
998,975,White Noise,4.00,2


**Вывод:** получено количество обзоров и средняя оценка для каждой из книг.

## Задание №3

**Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры**

In [13]:
query = '''
SELECT publisher, COUNT(*) as count
FROM 
    (SELECT *
     FROM books
     WHERE num_pages > 50) as bs_50
     
LEFT OUTER JOIN publishers AS pb ON bs_50.publisher_id = pb.publisher_id
GROUP BY pb.publisher
ORDER BY count DESC
LIMIT 1
'''

In [14]:
pd.io.sql.read_sql(query, con = engine) 

,publisher,count
0,Penguin Books,42


**Вывод:** издательство, выпустившее наибольшее количество книг - Penguin Books.

## Задание №4

**Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками**

In [15]:
query = '''
SELECT i.author, ROUND(AVG(i.rating_avg) OVER (PARTITION BY i.author), 2) AS avg_author_rating
FROM
    (SELECT u.bok_id, u.author, u.rating_avg
        FROM
            (SELECT bs.book_id as bok_id, *, ROUND(AVG(rg.rating) OVER (PARTITION BY au.author), 2) AS rating_avg
            FROM books as bs
            LEFT OUTER JOIN authors AS au ON bs.author_id = au.author_id
            LEFT OUTER JOIN ratings AS rg ON bs.book_id = rg.book_id
            WHERE bs.book_id IN

                -- Находим книги, количество отзывов которых больше 49 и группируем по book_id
                (SELECT bk.book_id
                FROM
                    -- Создаём таблицу с id книг и количеством оценок для каждой книги
                    (SELECT bs.book_id, COUNT(rg.rating) OVER (PARTITION BY bs.book_id) as count_rating
                    FROM books as bs
                    LEFT OUTER JOIN ratings AS rg ON bs.book_id = rg.book_id) as bk
            WHERE bk.count_rating > 49
            GROUP BY bk.book_id)) AS u
    GROUP BY u.bok_id, u.author, u.rating_avg) AS i
GROUP BY i.author, i.rating_avg
ORDER BY i.rating_avg DESC
LIMIT 1
'''

In [16]:
pd.io.sql.read_sql(query, con = engine) 

,author,avg_author_rating
0,J.K. Rowling/Mary GrandPré,4.29


**Вывод:** самый высоко оценённый автор - J.K. Rowling/Mary GrandPré.

## Задание №5

**Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок**

In [17]:
query = '''
SELECT ROUNd(AVG(cr.count_reviews), 0) as avg_count_reviews
FROM
    (SELECT username, COUNT(*) as count_reviews
    FROM reviews AS rv
    WHERE rv.username IN
        (SELECT us.username
            FROM

                (SELECT username, COUNT(*) as count_rating
                 FROM ratings
                 GROUP BY username) as us

            WHERE us.count_rating > 50
        )
    GROUP BY username) as cr
'''

In [18]:
pd.io.sql.read_sql(query, con = engine) 

,avg_count_reviews
0,24.0


**Вывод:** среднее количество обзоров от самых активных пользователей - 24.

# Общий вывод

После проведения анализа было полученно:
- что 819 книг вышло после 1-го января;
- количество обзоров и средняя оценка для каждой из книг;
- что издательство, выпустившее наибольшее количество книг - Penguin Books;
- что самый высоко оценённый автор - J.K. Rowling/Mary GrandPré;
- что среднее количество обзоров от самых активных пользователей - 24.